In [ ]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Optional, Literal, List

from functools import partial
import operator

from models.collect.collect import ModelCollect
from models.recommend.recommend import ModelRecommend
from models.qna.qna import ModelQna

from langchain_pinecone import PineconeVectorStore


# 정보 저장 state 선언 --------------------

class GraphState(TypedDict):
    messages: Annotated[list, add_messages]                         # 모든 메시지를 저장하는 리스트

    current_stage: Literal["collect", "recommend", "qna", "exit"]   # 현재 어떤 작업을 하고 있는지 저장

    collected_data: Optional[dict]                                  # 사용자에게서 모은 데이터(정보)를 저장하는 딕셔너리

    recommend_result: Annotated[Optional[List[str]], operator.add]  # 사용자에게 추천한 결과(해당 추천 결과는 재추천할때에 고려하지 않게 하기 위함)

    # None: 아무 행동도 하지 않음, Skip: 다음 단계로, Continue: 추천 만족, Retry: 추천 다시 받기, Restart: 처음부터 재시작, QnA: QnA로 이동
    user_action: Literal["None", "Skip", "Continue", "Retry", "Restart", "QnA",]

    picture_exist: Optional[str]
    

initial_state = {
    "messages": []

}
### tools 선언 ---------------------------
# tool 함수 선언
""" 
@tools
def tool_func(들어갈 인자들(타입 힌트 포함)) -> str:
    # RAG 수행
    return string 
"""

# tools 에는, 각각 이미지 처리 혹은 RAG를 수행하는 세가지 함수가 들어가야 함
tools = []

### 노드 선언 -----------------------------

def node_collect(state: GraphState, collector: ModelCollect):
    response, collected_data = collector.get_response(state["messages"], state["collected_data"])   # 어떤 정보를 전달했는지 알아야 하니까 collected_data도 같이 전달
    picture = collector.get_picture()   # 아마 get_picture에서 picture를 저장하고 해당 경로를 전달해야 할듯 (str)
    if picture is not None:
        return {
            "current_stage" : "collect",
            "messages": [response],  # 사용자에게 보여줄 질문
            "picture_exist": picture,                   # 사진이 있다면 사진의 저장 경로
            "collected_data": collected_data,            # 업데이트된 새로운 정보 딕셔너리
        }
    else:
        return {
            "current_stage" : "collect",
            "messages": [response],
            "picture_exist": "None",
            "collected_data": collected_data,
        }

def node_recommend(state: GraphState, recommender: ModelRecommend):
    response, recommend_result = recommender.get_response(state["messages"], state["collected_data"], state['recommend_result'])  
    
    # collected_data (정보를 저장한 딕셔너리) 도 같이 전달해주는 것이 낫지 않을지...   -> 어디에 전달하는 걸까요?
    # 사용자에게 보여줘야할 값 : response와, 추천 결과: recommend_result를 같이 반환해줘야 할듯 (추천 결과는 다시 추천 받을때 제외하기 위함)
    # collected_data: dict         # 사용자에게서 모은 데이터(정보)를 저장하는 딕셔너리
    # recommend_result: List[str]  # 사용자에게 추천한 결과(해당 추천 결과는 재추천할때에 고려하지 않게 하기 위함)
    return {
        "current_stage" : "recommend",
        "messages": [response],
        "recommend_result": [recommend_result],
    }

def node_qna(state: GraphState, chatbot: ModelQna):
    response = chatbot.get_response(state["messages"])

    return {
        "current_stage": "qna",
        "messages": [response],
    }

def node_end_state(state:GraphState):
    return {
        "current_stage": "exit"
    }


### router 선언 -----------------------

# 해당 router의 결과에 따라, 어떤 노드로 향할지 컨트롤
def main_router(state: GraphState):
    stage = state["current_stage"]
    last_message = state["messages"][-1]
    action = state["user_action"]

    if state["user_action"] == "Restart":
        return "restart"
    
    if stage == "collect":
        if ModelCollect.is_data_enough(state["collected_data"]):
            return "recommend"
        else:
            return "collect"
    
    elif stage == "recommend":
        if action == "Continue":
            return "exit"
        elif action == "QnA":
            return "qna"
        else:   # action == "Retry"
            return "recommend"
    
    elif stage == "qna":
        return "qna"
    
    elif stage == "exit":
        return "exit"
    
def is_tool_calls(state: GraphState):
    last_message = state["messages"][-1]

    if last_message.tool_calls:
        return "tool_call"
    else:
        return "done"
    
def tool_back_to_caller(state: GraphState) -> str:
    current_state = state.get("current_state")

    if current_state and current_state in ["collect", "recommend", "qna"]:
        return current_state
    
    return "done"


### workflow 구현----------------------

model_collect = ModelCollect(tools)
model_recommend = ModelRecommend(tools)
model_qna = ModelQna(tools)

workflow = StateGraph(GraphState)

workflow.add_node("collect", partial(node_collect, collector=model_collect))
workflow.add_node("recommend", partial(node_collect, collector=model_recommend))
workflow.add_node("qna", partial(node_collect, collector=model_qna))
workflow.add_node("exit", node_end_state)
workflow.add_node("rag_tool", ToolNode(tools))

workflow.set_entry_point("__start__")

workflow.add_edge("exit", END)
workflow.add_edge

workflow.add_conditional_edges(
    "__start__",
    main_router,
    {
        "collect": "collect",
        "recommend": "recommend",
        "qna": "qna",
        "done": END,
        "exit": "exit"
    }
)

workflow.add_conditional_edges(
    "collect",
    is_tool_calls,
    {
        "tool_call": "rag_tool",
        "done": END,
    }
)

workflow.add_conditional_edges(
    "recommend",
    is_tool_calls,
    {
        "tool_call": "rag_tool",
        "done": END,
    }
)

workflow.add_conditional_edges(
    "qna",
    is_tool_calls,
    {
        "tool_call": "rag_tool",
        "done": END,
    }
)

workflow.add_conditional_edges(
    "rag_tool",
    tool_back_to_caller,
    {
        "collect": "collect",
        "recommend": "recommend",
        "qna": "qna",
        "exit": END,
    }
)

### 그래프 컴파일 ----------------------
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

### 실제 구동 코드 ---------------------
def run_chat_loop(app, memory: MemorySaver, initial_state: dict):
    thread_id_01 = "basic_user"
    config = {"configurable": {"thread_id": thread_id_01}}

    response = app.invoke(initial_state)

    while True:
        try:
            current_state = memory.get_state(config).values
            message = current_state["messages"][-1]

            print("="*30)
            print(f"채팅 시작: 현재 작업 {current_state['current_stage']}")
            print(f"AI   : {message}")
            print("="*30)
            user_input = input("User : ")

            if user_input.lower() == "종료":
                print("종료합니다...")
                break

            input_delta = {
                "messages": [HumanMessage(content=user_input)],
                "user_action": "None",
            }

            # 이후 확장성을 위해 app.stream 사용
            stream_generator = app.stream(input_delta, config=config)

            print("생성중", end='')
            for step in stream_generator:
                print('.', end=' ', flush=True)
        except Exception as e:
            print(f"오류 발생: {e}")
            break

### -----------------------------------

if __name__ == "__main__":
    run_chat_loop(app, memory, initial_state)


In [ ]:
import json
from typing import List, Optional

c:\Users\tndl3\anaconda3\envs\project3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
collected_data = {
    "purpose": "거실 / 거실의 빈 벽이 너무 심심해서 놓고 싶어",
    "preferred_style": "미니멀",
    "preferred_color": "#A1B07C",
    "plant_type": '난',
    "season": "여름",
    "humidity": "건조",
    "has_dog": True,
    "has_cat": False,
    "isAirCond": True,
    "watering_frequency": "주1회 가능",
    "user_experience": "초보",
    "emotion": "위로",
    "description":"설명~",
}

In [ ]:
# 1. node_recommend()
# 2. (rag 전) get_response(): tool 호출 
# 3. tool recommend_rag(): 벡터DB에서 검색 후 반환
# 4. (rag 후) get_response(): 검색된 데이터를 바탕으로 LLM이 응답 생성
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage

class ModelRecommend:
    def __init__(self, tools):
        self.tools = tools

    def get_response(self, messages, collected_data, prev_results: Optional[List[str]]):        
        prompt= f"""
            ### 요구사항 ###
            {collected_data}

            ### 이미 추천한 식물 (다시 추천하지 않습니다) ###
            {prev_results}

            ### 설명 ###
            당신은 친절한 식물 전문가입니다. 한국어로 친절하게 답변하세요.
            RAG 검색 결과를 참고해서 사용자에게 식물 1가지를 추천하세요.
            이미 추천한 식물은 사용자가 거부한 식물이니 추천하지 않습니다.

            ### 응답목록 ###
            - 추천하는 식물 이름
            - 추천하는 이유
            - 식물의 꽃말
            - 식물을 키우는 법
            - 식물의 특징

            ### 출력형식 ###
            {{
                "flowNm": "",
                "response": ""
            }}
        """
        
        system_msg = SystemMessage(prompt)
        input_msg = [system_msg] + messages
            
        model = ChatOpenAI(
            model='gpt-4o-mini',
            temperature=1
        ).bind_tools(self.tools)
            
        response = model.invoke(input_msg)
        recommend_result = ""
        if response.content != '':
            res_json = json.loads(response.content)
            recommend_result = res_json['flowNm']
            
        return response, recommend_result

In [ ]:
@tool
def tool_rag_qna(query: str) -> str:
    """식물 상담 QnA 전용 RAG 도구"""
    
    #----- 전역에서 한번만 바꿔야 할듯한

    ##-----

    docs = retriever.invoke(query) # retriever(query)
    return "\n\n".join([d.page_content for d in docs])


tools = [tool_rag_qna]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
@tool
def tool_rag_recommend(query: str) -> str:
    """식물 추천 전용 RAG 도구"""
    vector_store = PineconeVectorStore(
        index=index, 
        embedding=OpenAIEmbeddings(model="text-embedding-3-small")
    )
    
    retriever = vector_store.as_retriever(search_kwargs={"k": 1})
    retrievals = retriever.batch([query])
    
    return str(retrievals[0])